In [1]:
# FETCH PACKAGES
!pip install pyTigerGraph
!pip install python-dotenv
import pyTigerGraph as tg
from dotenv import load_dotenv
import os
import pandas as pd
import json
import requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 2.2 MB/s eta 0:00:00


In [2]:
#GRAB DATA FILES. CREATE DATA FRAMES FROM .CSV FILES AND PREPARE FOR UPSERTION
!git clone https://github.com/ArshdeepKaur/knowledge_graph_using_tigergraph

#VERTEX: movie
df_movies = pd.read_csv("/content/knowledge_graph_using_tigergraph/data/movies.csv", sep=";")
null_df = df_movies[df_movies.isna().any(axis=1)]
print(null_df)
df_movies['tagline']  = df_movies['tagline'].fillna('') #tagline (str) has NaN
print(df_movies.info())
print(df_movies.head(2))

#VERTEX: person
dtype_dict = {
    'name': str,
    'born': int}
df_actors = pd.read_csv("/content/knowledge_graph_using_tigergraph/data/actors.csv", sep=";")
null_df = df_actors[df_actors.isna().any(axis=1)]
print(null_df)
df_actors['born'] = df_actors['born'].fillna(0) #born (int) has NaN
df_actors = df_actors.astype(dtype_dict)
print(df_actors.info())
print(df_actors.head(2))

#EDGE ACTED_IN
df_acted_in = pd.read_csv("/content/knowledge_graph_using_tigergraph/data/acted_in.csv", sep=";")
print(df_acted_in.info())
print(df_acted_in.head(2))


Cloning into 'knowledge_graph_using_tigergraph'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 4), reused 15 (delta 3), pack-reused 0
Receiving objects: 100% (19/19), 9.91 KiB | 3.30 MiB/s, done.
Resolving deltas: 100% (4/4), done.
                     title tagline  released
33  Something's Gotta Give     NaN      2003
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     38 non-null     object
 1   tagline   38 non-null     object
 2   released  38 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ KB
None
                 title                    tagline  released
0           The Matrix  Welcome to the Real World      1999
1  The Matrix Reloaded             Free your mind      2003
                 name  born
103     N

In [3]:
#CREATE HOST STRING USING SUBDOMAIN
load_dotenv('/content/env', override=True)

TG_IP = os.getenv('TG_IP')
TG_CLUSTER = os.getenv('TG_CLUSTER')
TG_SUBDOMAIN = os.getenv('TG_SUBDOMAIN')
TG_DOMAIN=f"{TG_SUBDOMAIN}.i.tgcloud.io"
TG_SUBNET = os.getenv('TG_SUBNET')
TG_USERNAME = os.getenv('TG_USERNAME')
TG_PASSWORD = os.getenv('TG_PASSWORD')
TG_GRAPH = os.getenv('TG_GRAPH')
host_string = f"https://{TG_DOMAIN}"

In [4]:
# CONNECT TO TigerGraph CLOUD
conn = tg.TigerGraphConnection(host=host_string, username=TG_USERNAME, password=TG_PASSWORD)
print("Connected")
print(conn)

Connected


In [5]:
#DANGEROUS! - CLEAR THE SCHEMA AND GRAPH
conn.gsql('''
USE GLOBAL
DROP ALL
''')

'Dropping all, about 1 minute ...\nAbort all active loading jobs\nTry to abort all loading jobs on graph movie_graph, it may take a while ...\nResetting GPE...\nSuccessfully reset GPE and GSE\nStopping GPE GSE\nSuccessfully stopped GPE GSE in 0.005 seconds\nClearing graph store...\nSuccessfully cleared graph store\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.078 seconds\nEverything is dropped.'

In [6]:
#CREATE VERTICES & EDGES
conn.gsql('''
USE GLOBAL
CREATE VERTEX person (
  PRIMARY_ID name STRING, born INT
)
WITH primary_id_as_attribute="true"

CREATE VERTEX movie (
  PRIMARY_ID title STRING, tagline STRING, released INT
)
WITH primary_id_as_attribute="true"

CREATE UNDIRECTED EDGE ACTED_IN (FROM person, TO movie)

CREATE UNDIRECTED EDGE DIRECTED (FROM person, TO movie)

CREATE UNDIRECTED EDGE FRIEND (FROM person, TO person)
''')

'Successfully created vertex types: [person].\nSuccessfully created vertex types: [movie].\nSuccessfully created edge types: [ACTED_IN].\nSuccessfully created edge types: [DIRECTED].\nSuccessfully created edge types: [FRIEND].'

In [7]:
#CREATE GRAPH
conn.gsql('''
CREATE GRAPH movie_graph(person, movie, ACTED_IN, DIRECTED, FRIEND)
''')

'Stopping GPE GSE RESTPP\nSuccessfully stopped GPE GSE RESTPP in 31.616 seconds\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.081 seconds\nThe graph movie_graph is created.'

In [8]:
#FETCH ACCESS TOKEN
conn.graphname = TG_GRAPH
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)

print(conn, conn.graphname)

#VERIFY THE SCHEMA
print(conn.getSchema())

print(conn.gsql("LS"))
print(conn.getVertexTypes())
print(conn.getEdgeTypes())

<pyTigerGraph.pyTigerGraph.TigerGraphConnection object at 0x7c3a42926410> movie_graph
{'GraphName': 'movie_graph', 'VertexTypes': [{'Config': {'STATS': 'OUTDEGREE_BY_EDGETYPE', 'PRIMARY_ID_AS_ATTRIBUTE': True}, 'Attributes': [{'AttributeType': {'Name': 'INT'}, 'AttributeName': 'born'}], 'PrimaryId': {'AttributeType': {'Name': 'STRING'}, 'PrimaryIdAsAttribute': True, 'AttributeName': 'name'}, 'Name': 'person'}, {'Config': {'STATS': 'OUTDEGREE_BY_EDGETYPE', 'PRIMARY_ID_AS_ATTRIBUTE': True}, 'Attributes': [{'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'tagline'}, {'AttributeType': {'Name': 'INT'}, 'AttributeName': 'released'}], 'PrimaryId': {'AttributeType': {'Name': 'STRING'}, 'PrimaryIdAsAttribute': True, 'AttributeName': 'title'}, 'Name': 'movie'}], 'EdgeTypes': [{'IsDirected': False, 'ToVertexTypeName': 'movie', 'Config': {}, 'Attributes': [], 'FromVertexTypeName': 'person', 'Name': 'ACTED_IN'}, {'IsDirected': False, 'ToVertexTypeName': 'movie', 'Config': {}, 'Attributes': []

In [9]:
#UPSERT DATA INTO VERTEX: movie
v_movies = conn.upsertVertexDataFrame(df_movies, 'movie', v_id='title', attributes={'title':'title', 'tagline': 'tagline', 'released': 'released'})
print(str(v_movies) + " movie VERTICES upserted")

38 movie VERTICES upserted


In [10]:
#UPSERT DATA INTO VERTEX: person
v_actors = conn.upsertVertexDataFrame(df_actors, 'person', v_id='name', attributes={'name':'name', 'born': 'born'})
print(str(v_actors) + " person VERTICES upserted")

133 person VERTICES upserted


In [11]:
#UPSERT DATA INTO EDGE: ACTED_IN
e_ACTED_IN = conn.upsertEdgeDataFrame(df=df_acted_in, sourceVertexType='person', edgeType="ACTED_IN", targetVertexType='movie', from_id='actor', to_id='movie', attributes={}
)
print(str(e_ACTED_IN) + " ACTED_IN EDGES upserted")


172 ACTED_IN EDGES upserted


In [14]:
# SIMPLE SELECT QUERY
query = '''
    USE GRAPH movie_graph
    DROP QUERY view_all_movies

    CREATE QUERY view_all_movies () {
        result = SELECT m FROM movie:m;
        PRINT result[result.title, result.tagline];
    }
    INSTALL QUERY view_all_movies
    '''
results = conn.gsql(query)
print (results)

results = conn.runInstalledQuery("view_all_movies")
print(json.dumps(results, indent=2))


[
  {
    "result": [
      {
        "v_id": "A Few Good Men",
        "v_type": "movie",
        "attributes": {
          "result.title": "A Few Good Men",
          "result.tagline": "In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth."
        }
      },
      {
        "v_id": "Charlie Wilson's War",
        "v_type": "movie",
        "attributes": {
          "result.title": "Charlie Wilson's War",
          "result.tagline": "A stiff drink. A little mascara. A lot of nerve. Who said they couldn't bring down the Soviet empire."
        }
      },
      {
        "v_id": "Cast Away",
        "v_type": "movie",
        "attributes": {
          "result.title": "Cast Away",
          "result.tagline": "At the edge of the world, his journey begins."
        }
      },
      {
        "v_id": "The Matrix Reloaded",
        "v_type": "movie",
        "attributes": 

In [15]:
# SIMPLE 1-HOP PARAMETERIZED QUERY
query = '''
    USE GRAPH movie_graph
    DROP QUERY select_all_movies_for_person
    CREATE QUERY select_all_movies_for_person (VERTEX<person> p) {
        start = {p};
        result = SELECT tgt
            FROM start:s -(ACTED_IN:e)- movie:tgt;
        PRINT result[result.title];
    }
INSTALL QUERY select_all_movies_for_person
'''

results = conn.gsql(query)
print(results)

parameters = {
    "p": "Tom Hanks"
}
results = conn.runInstalledQuery("select_all_movies_for_person", params=parameters)
print(json.dumps(results, indent=2))

[
  {
    "result": [
      {
        "v_id": "Charlie Wilson's War",
        "v_type": "movie",
        "attributes": {
          "result.title": "Charlie Wilson's War"
        }
      },
      {
        "v_id": "Cast Away",
        "v_type": "movie",
        "attributes": {
          "result.title": "Cast Away"
        }
      },
      {
        "v_id": "The Polar Express",
        "v_type": "movie",
        "attributes": {
          "result.title": "The Polar Express"
        }
      },
      {
        "v_id": "Apollo 13",
        "v_type": "movie",
        "attributes": {
          "result.title": "Apollo 13"
        }
      },
      {
        "v_id": "A League of Their Own",
        "v_type": "movie",
        "attributes": {
          "result.title": "A League of Their Own"
        }
      },
      {
        "v_id": "Sleepless in Seattle",
        "v_type": "movie",
        "attributes": {
          "result.title": "Sleepless in Seattle"
        }
      },
      {
        "v_id": 

In [25]:
# 2-HOP PARAMETERIZED QUERY WITH ACCUMULATORS
query = '''
USE GRAPH movie_graph

DROP QUERY movies_of_costars

CREATE QUERY movies_of_costars (VERTEX<person> p, INT current_year = 2024) {
    OrAccum  @visited = FALSE;
    SumAccum <INT> @@count_movies;
    SumAccum <INT> @@count_costars;
    AvgAccum @@avg_age;
    ListAccum<STRING> @movies;
    ListAccum<STRING> @costars;

    start = {p};

    first_neighbors = SELECT tgt_m
        FROM start:s -(ACTED_IN:e)- movie:tgt_m
        ACCUM tgt_m.@visited += TRUE, s.@visited += TRUE, tgt_m.@movies += tgt_m.title
        POST-ACCUM (tgt_m) @@count_movies += 1;

    second_neighbors = SELECT tgt_p
        FROM person:tgt_p -(:e)- first_neighbors
        WHERE tgt_p.@visited == FALSE
        POST-ACCUM (tgt_p) @@count_costars += 1, @@avg_age += (current_year - tgt_p.born), tgt_p.@costars += tgt_p.name;

    PRINT "titles";
    PRINT first_neighbors[first_neighbors.title];
    PRINT "titles using @movies accumulator";
    PRINT first_neighbors[first_neighbors.@movies];
    PRINT "costars";
    PRINT second_neighbors[second_neighbors.name];
    PRINT "costars using @costars accumulator";
    PRINT second_neighbors[second_neighbors.@costars];
    PRINT "count of movies";
    PRINT @@count_movies;
    PRINT "count of costars";
    PRINT @@count_costars;
    PRINT "average age of costars";
    PRINT @@avg_age;
}

INSTALL QUERY movies_of_costars
'''

results = conn.gsql(query)
print(results)

parameters = {
    "p": "Tom Hanks"
}
results = conn.runInstalledQuery("movies_of_costars", params=parameters)
print(json.dumps(results, indent=2))

Using graph 'movie_graph'
Successfully dropped queries on the graph 'movie_graph': [movies_of_costars].
Warning in a transformed block of query movies_of_costars (WARN-2): line 25, col 35
unsatisfiable pattern _>
Warning in a transformed block of query movies_of_costars (WARN-2): line 25, col 38
unsatisfiable pattern <_
Warning in a transformed block of query movies_of_costars (WARN-7): line 28, col 1
Non-binding POST-ACCUM clauses will be deprecated soon. Please rewrite the
clause as: POST-ACCUM (selectAlias) ...
Successfully created queries: [movies_of_costars].
Start installing queries, about 1 minute ...
movies_of_costars query: curl -X GET 'https://127.0.0.1:9000/query/movie_graph/movies_of_costars?p=VALUE&[current_year=VALUE]'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

[                                                                                     ] 0% (0/1) 

In [ ]:
#CREATE LOADING JOBs
results = conn.gsql('''
USE GRAPH movie_graph
BEGIN
DROP JOB py_job_movies
CREATE LOADING JOB py_job_movies FOR GRAPH movie_graph {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX movie VALUES($0, $1, $2) USING SEPARATOR=";", HEADER="true", EOL="\n", QUOTE="double";
    }
END''')
print(results)

results = conn.gsql('''
USE GRAPH movie_graph
BEGIN
DROP JOB py_job_persons
CREATE LOADING JOB py_job_persons FOR GRAPH movie_graph {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX person VALUES($0, $1) USING SEPARATOR=";", HEADER="true", EOL="\n", QUOTE="double";
    }
END''')
print(results)


results = conn.gsql('''
USE GRAPH movie_graph
BEGIN
DROP JOB py_job_acted_in
CREATE LOADING JOB py_job_acted_in FOR GRAPH movie_graph {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE ACTED_IN VALUES($0, $1) USING SEPARATOR=";", HEADER="true", EOL="\n", QUOTE="double";
    }
END''')
print(results)

print(conn.graphname)
conn.gsql("USE GRAPH movie_graph")
results = conn.gsql('''USE GRAPH movie_graph
    SHOW JOB *''')
print(results)

print(conn.gsql("LS"))

Using graph 'movie_graph'
Successfully dropped jobs on the graph 'movie_graph': [py_job_movies].
Successfully created loading jobs: [py_job_movies].
Using graph 'movie_graph'
Successfully dropped jobs on the graph 'movie_graph': [py_job_persons].
Successfully created loading jobs: [py_job_persons].
Using graph 'movie_graph'
Successfully dropped jobs on the graph 'movie_graph': [py_job_acted_in].
Successfully created loading jobs: [py_job_acted_in].
movie_graph
Using graph 'movie_graph'
  - CREATE LOADING JOB py_job_movies FOR GRAPH movie_graph {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX movie VALUES($0, $1, $2) USING SEPARATOR=";", HEADER="true", EOL="
", QUOTE="double";
    }

  - CREATE LOADING JOB py_job_persons FOR GRAPH movie_graph {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX person VALUES($0, $1) USING SEPARATOR=";", HEADER="true", EOL="
", QUOTE="double";
    }

  - CREATE LOADING JOB py_job_acted_in FOR GRAPH movie_graph {


In [ ]:
#RUN LOADING JOBs
dataSource = "/content/knowledge_graph_using_tigergraph/data/movies.csv"
results = conn.gsql('''
USE GRAPH movie_graph
RUN LOADING JOB py_job_movies USING MyDataSource="'+dataSource+'"
''')
print(results)

TigerGraphException: Using graph 'movie_graph'
Semantic Check Fails: TigerGraph sensitive directory '/home/tigergraph/tigergraph/app' is not allowed in path '/home/tigergraph/tigergraph/app/3.10.0/dev/gdk/gsql/'+dataSource+''. Please use another path.

In [ ]:
#RUN LOADING JOBs
f_movies =  "/content/knowledge_graph_using_tigergraph/data/movies.csv"
f_persons = "/content/knowledge_graph_using_tigergraph/data/actors.csv"
f_acted_in = "/content/knowledge_graph_using_tigergraph/data/acted_in.csv"

res = conn.runLoadingJobWithFile(filePath=f_movies, fileTag='file_movies', jobName='py_job_movies', sep=";")
print(json.dumps(res, indent=2))
res = conn.runLoadingJobWithFile(filePath=f_persons, fileTag='file_persons', jobName='py_job_persons', sep=";")
print(json.dumps(res, indent=2))
res = conn.runLoadingJobWithFile(filePath=f_acted_in, fileTag='file_acted_in', jobName='py_job_acted_in', sep=";")
print(json.dumps(res, indent=2))


TigerGraphException: ('Exception in OnFinish: GetResult index out of range, index: 0 | response_datas_.size()0', 'REST-10005')